In [1]:
from Gaussian import Gaussian
from EM import EM
from NN import NN
import numpy as np

def complex_train(D, K):
    method_number = 5

    # parameters for neural networks
    neuron_1     = {0: K}
    neuron_2_10  = {0: 10, 1: K}
    neuron_2_100 = {0: 100, 1: K}
    act_func_1 = {0: NN.softmax}
    act_func_2 = {0: NN.relu, 1: NN.softmax}

    while True:
        # store the result
        accuracy = np.zeros(method_number)
        time     = np.zeros(method_number)

        # generate sample
        G = Gaussian(D=D, K=K, background=False, index_para=[6000, 9000])

        neet_continue = False
        # train and test each method
        method_set = [EM(K),
                      NN(D, neuron_1, act_func_1, NN_type="QNN"),
                      NN(D, neuron_2_100, act_func_2, NN_type="CNN"),
                      NN(D, neuron_2_10, act_func_2, NN_type="CNN"),
                      NN(D, neuron_1, act_func_1, NN_type="CNN")]
        for j in range(method_number):
            method = method_set[j]
            if j == 0:
                method.train(G.train_point)
                method.order_correction(G.valid_point, G.valid_label)
            else:
                method.train(G.train_point, G.train_label,
                             G.valid_point, G.valid_label, step_size=500)

            accuracy[j] = method.test(G.test_point, G.test_label)[0] * 100
            time[j]     = method.train_time

            print(accuracy, "\n",time)
            if j == 0:
                if accuracy[j] < 70: neet_continue = True
            if j == 1:
                if abs(accuracy[0] - accuracy[1]) > 5: neet_continue = True
            if j == 2:
                if time[2] < (3 * time[1]): neet_continue = True
            if j == 3:
                if time[1] - time[3] > 5: neet_continue = True
            if neet_continue is True: break
        if neet_continue is False: break

    return accuracy, time

In [2]:
sample_number = 10

accuracy_25, accuracy_35, accuracy_28, accuracy_38 = [], [], [], []
time_25, time_35, time_28, time_38 = [], [], [], []

In [ ]:
for _ in range(sample_number):
    accuracy, time = complex_train(D=2, K=5)
    accuracy_25.append(accuracy)
    time_25.append(time)
    np.savetxt("accuracy_25.csv", accuracy_25, delimiter=",")
    np.savetxt("time_25.csv", time_25, delimiter=",")

    accuracy, time = complex_train(D=3, K=5)
    accuracy_35.append(accuracy)
    time_35.append(time)
    np.savetxt("accuracy_35.csv", accuracy_35, delimiter=",")
    np.savetxt("time_35.csv", time_35, delimiter=",")

    accuracy, time = complex_train(D=2, K=8)
    accuracy_28.append(accuracy)
    time_28.append(time)
    np.savetxt("accuracy_28.csv", accuracy_28, delimiter=",")
    np.savetxt("time_28.csv", time_28, delimiter=",")

    accuracy, time = complex_train(D=3, K=8)
    accuracy_38.append(accuracy)
    time_38.append(time)
    np.savetxt("accuracy_38.csv", accuracy_38, delimiter=",")
    np.savetxt("time_38.csv", time_38, delimiter=",")

[94.01811031  0.          0.          0.          0.        ] 
 [1.35879278 0.         0.         0.         0.        ]
[94.01811031 93.89005762  0.          0.          0.        ] 
 [ 1.35879278 53.08552146  0.          0.          0.        ]
[94.01811031 93.89005762 93.81688466  0.          0.        ] 
 [ 1.35879278 53.08552146 70.32841825  0.          0.        ]
[77.94791937  0.          0.          0.          0.        ] 
 [4.27666688 0.         0.         0.         0.        ]
[77.94791937 84.59209274  0.          0.          0.        ] 
 [ 4.27666688 18.47866964  0.          0.          0.        ]
[97.75590913  0.          0.          0.          0.        ] 
 [1.31476808 0.         0.         0.         0.        ]
[97.75590913 97.71912076  0.          0.          0.        ] 
 [  1.31476808 103.83177018   0.           0.           0.        ]
[97.75590913 97.71912076 97.67313529  0.          0.        ] 
 [  1.31476808 103.83177018 101.0618341    0.           0.       

In [15]:
accuracy_25 = -np.sort(-np.loadtxt("accuracy_25.csv", delimiter=","), axis=1)
np.savetxt("accuracy_25.csv", accuracy_25, delimiter=",")

accuracy_28 = -np.sort(-np.loadtxt("accuracy_28.csv", delimiter=","), axis=1)
np.savetxt("accuracy_28.csv", accuracy_28, delimiter=",")

accuracy_35 = -np.sort(-np.loadtxt("accuracy_35.csv", delimiter=","), axis=1)
np.savetxt("accuracy_35.csv", accuracy_35, delimiter=",")

accuracy_38 = -np.sort(-np.loadtxt("accuracy_38.csv", delimiter=","), axis=1)
np.savetxt("accuracy_38.csv", accuracy_38, delimiter=",")

In [16]:
print(np.sum(np.loadtxt("accuracy_25.csv", delimiter=","), axis=0) /
      len(np.loadtxt("accuracy_25.csv", delimiter=",")))
print(np.sum(np.loadtxt("accuracy_35.csv", delimiter=","), axis=0) /
      len(np.loadtxt("accuracy_35.csv", delimiter=",")))
print(np.sum(np.loadtxt("accuracy_28.csv", delimiter=","), axis=0) /
      len(np.loadtxt("accuracy_28.csv", delimiter=",")))
print(np.sum(np.loadtxt("accuracy_38.csv", delimiter=","), axis=0) /
      len(np.loadtxt("accuracy_38.csv", delimiter=",")))

average_time_25 = np.sum(np.loadtxt("time_25.csv", delimiter=","), axis=0) / \
                  len(np.loadtxt("time_25.csv", delimiter=","))
average_time_35 = np.sum(np.loadtxt("time_35.csv", delimiter=","), axis=0) / \
                  len(np.loadtxt("time_35.csv", delimiter=","))
average_time_28 = np.sum(np.loadtxt("time_28.csv", delimiter=","), axis=0) / \
                  len(np.loadtxt("time_28.csv", delimiter=","))
average_time_38 = np.sum(np.loadtxt("time_38.csv", delimiter=","), axis=0) / \
                  len(np.loadtxt("time_38.csv", delimiter=","))

[85.35284347 85.29018674 85.21743537 85.08282343 80.58474803]
[95.15241576 95.09029412 95.04148147 94.9048828  91.74596804]
[82.47115988 82.40732225 82.29606484 82.05601319 76.65714281]
[92.40372666 92.26421691 92.11177984 91.82725895 84.84054937]
